<a href="https://colab.research.google.com/github/csce585-mlsystems/CSCE585ProjectROI/blob/michelleBranch/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> Running experiments in ModelDevelopment Directory



Cloned the repository

In [ ]:
!git clone https://github.com/csce585-mlsystems/CSCE585ProjectROI

Cloning into 'CSCE585ProjectROI'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 526 (delta 90), reused 150 (delta 52), pack-reused 304 (from 1)
Receiving objects: 100% (526/526), 1.36 MiB | 10.15 MiB/s, done.
Resolving deltas: 100% (231/231), done.


After cloning, you might need to navigate into the repository's directory. Then, install any necessary Python packages. Often, these are listed in a `requirements.txt` file. You may need to check the repository's documentation for the exact file name or commands.

In [ ]:
# Repo directories
import os
os.chdir('CSCE585ProjectROI')


Here I'm uploading my dependencies

In [ ]:
!pip install reactpy reactpy-router flask llama-cpp-python TensorFlow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.3/110.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4422346 sha256=376e4dd75e85b2c914f4f89cd62d2e0b222ccfe4b9e2b58da0dcbc25922ff98c
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python


In [ ]:
import sys
print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


Verified that the notebook is running from python 3.12.12

Finally, you can load the model using the code provided by your classmate. The exact method will depend on how the model was saved (e.g., using `pickle`, `torch.load`, `tf.keras.models.load_model`). You will likely need to import specific libraries (like TensorFlow, PyTorch, or scikit-learn) depending on how the model was built.

Uploading Specific libaries

In [37]:
# Body of neccessary imports
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np #<-- May be optional not sure as of 10/13/25.
# import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist

Uploaded the model from local computer

In [1]:
from google.colab import files
model = files.upload()

Saving set1.py to set1.py


In [14]:
from google.colab import files
apple = files.upload()

Saving apple_Metrics.csv to apple_Metrics (1).csv


In [15]:
from google.colab import files
google = files.upload()

Saving google_Metrics.csv to google_Metrics (1).csv


Step 1: Split data into training group, validation and test sets

X : Features

y : Target variable of dataset

In [16]:
import numpy as np
import pandas as pd
import io
from sklearn.model_selection import train_test_split

In [23]:
for fn in apple.keys():
    apple = pd.read_csv(io.StringIO(apple[fn].decode('utf-8')))

In [24]:
for fn in google.keys():
  google = pd.read_csv(io.StringIO(google[fn].decode('utf-8')))

Here I'm defining X and y values for the Apple dataset where my metrics will be X values and dates will be y value

To extract my X values I defined each row value ex Operating Cash Flow as an input as a keras layer

Then I processed each input independently

In [48]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

input_1 = keras.Input(shape=(2,), name="Operating Cash Flow")
input_2 = keras.Input(shape=(3,), name="Capital Expenditure")
input_3 = keras.Input(shape=(4,), name = "Free Cash Flow")
input_4 = keras.Input(shape=(5,), name = "Total Debt")
input_5 = keras.Input(shape=(6,), name = "Common Stockholder Equity")
input_6 = keras.Input(shape=(7,), name = 'Total Liabilities Net Minority Interest')
input_7 = keras.Input(shape=(8,), name = 'Total Assets')
input_8 = keras.Input(shape=(9,), name = 'Shares Outstanding')
input_9 = keras.Input(shape=(10,), name = 'Net Income Common Stockholders')
input_10 = keras.Input(shape=(11,), name = 'Diluted Average Shares')
input_11 = keras.Input(shape=(12,), name ='Diluted EPS')
input_12 = keras.Input(shape=(13,), name = 'Price')

In [51]:
x1 = layers.Dense(64, activation="relu")(input_1)
x2 = layers.Dense(64, activation="relu")(input_2)
x3 = layers.Dense(64, activation="relu")(input_3)
x4 = layers.Dense(64, activation="relu")(input_4)
x5 = layers.Dense(64, activation="relu")(input_5)
x6 = layers.Dense(64, activation="relu")(input_6)
x7 = layers.Dense(64, activation="relu")(input_7)
x8 = layers.Dense(64, activation="relu")(input_8)
x9 = layers.Dense(64, activation="relu")(input_9)
x10 = layers.Dense(64, activation="relu")(input_10)
x11 = layers.Dense(64, activation="relu")(input_11)
x12 = layers.Dense(64, activation="relu")(input_12)

Here I merged all the inputs into one single X value

In [54]:
X = layers.concatenate([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12])

Now for the y values we have to determine our target values



In [56]:
import yfinance as yf
from datetime import timedelta

In [61]:
def fetch_market_data(self, start_date="2024-06-24"):
        """
        Fetches historical price data for stocks and benchmarks.
        """
        print(f"Fetching data for {len(self.tickers)} stocks and benchmarks...")

        # 1. Fetch Benchmarks
        bench_data = yf.download(self.benchmark_tickers, start=start_date, progress=False)['Close']
        bench_data = bench_data.ffill() # Fill missing data (holidays)

        # 2. Fetch Stocks
        stock_data = yf.download(self.tickers, start=start_date, progress=False)['Close']
        stock_data = stock_data.ffill()

        print(stock_data, bench_data)

        return stock_data, bench_data

In [22]:
#Data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Step 2: Use data leakage prevention techniques such as walk forward cross validation

Step 3: Hypertune the parameters using the validation data set

Since we are measuring numerical values it would be best to use regression based evaluations such as MSE